In [4]:
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import MathsUtilities as MUte
%matplotlib inline

ModuleNotFoundError: No module named 'MathsUtilities'

In [ ]:
Colors = ['darkblue',           
'darkcyan',           
'darkgoldenrod',      
'darkgray',           
'darkgreen',          
'darkkhaki',          
'darkmagenta',        
'darkolivegreen',     
'darkorange',         
'darkorchid',         
'darkred',            
'darkviolet',
'blue',           
'cyan',           
'goldenrod',      
'gray',           
'green',
'black']
Markers = ['o','s','^']

In [ ]:
con = sqlite3.connect(r'C:\Users\jjojeda\Dropbox\APSIMPotato\Potato.db')

In [ ]:
HarvestObsPred = pd.read_sql("Select * from FinalYieldData",
                        con)
#HarvestObsPred.set_index('SimulationID',inplace=True)

In [ ]:
HarvestObsPred

In [ ]:
Factors = pd.read_sql("Select * from _Factors",
                        con)
ValidationIndex = Factors.drop_duplicates(subset=['SimulationName']).loc[:,Factors.columns.isin(['SimulationID','ExperimentName','SimulatinName','FolderName'])]
Experiments = ValidationIndex.ExperimentName.drop_duplicates().tolist()
Folders = ValidationIndex.FolderName.drop_duplicates().tolist()

In [ ]:
HarvestObsPred.columns

In [ ]:
obs=HarvestObsPred.loc[:,'Observed.Potato.TotalLive.Wt']
pre=HarvestObsPred.loc[:,'Predicted.Potato.TotalLive.Wt']

In [ ]:
plt.plot(obs,pre,"o",color="r")
plt.plot([0,2000],[0,2000],"--",color="k")

In [ ]:
list(HarvestObsPred.columns)

In [ ]:
pd.options.display.max_rows = 999

In [ ]:
Factors.sort_values(by='SimulationID')

In [ ]:
Experiments

In [ ]:
def MakeLabel(RegStats):
    #text = RegStats.Name + '  n = ' + str(RegStats.n)
    text = '\ny = ' + '%.2f'%RegStats.Intercept + '(se ' + '%.2f'%RegStats.SEintercept + ') + ' + '%.2f'%RegStats.Slope + '(se ' + '%.2f'%RegStats.SEslope + ') x' 
    text += '\n$r^2$ =' + '%.2f'%RegStats.R2 + ' RMSE = ' + '%.2f'%RegStats.RMSE +' NSE = ' + '%.2f'%RegStats.NSE
    text += '\nME = ' + '%.2f'%RegStats.ME + ' MAE = ' + '%.2f'%RegStats.MAE
    text += '\nn = ' + str(RegStats.n)
    return text

In [ ]:
HarvestObsPred

In [ ]:
Graph = plt.figure(figsize=(15,15))
Variables = ['AboveGround.Wt','Grain.Number','Grain.Wt',
             'AboveGround.N','Grain.N','Grain.Protein',
             'Phenology.FlagLeafDAS.Value()','Phenology.FloweringDAS.Value()']
Units = ['$g/m^3$','$/m^3$','$g/m^3$',
        '$g/m^3$','$g/m^3$','$g/m^3$',
        'days','days']
Names = ['Biomass','Grain Number', 'Yield', 'Biomass N', 'Grain N',
         'Grain Protein', 'Flag Leaf','Anthesis']
Pos = 1
for Var in Variables:
    NaNFilter = np.isnan(HarvestObsPred.loc[:,'Predicted.Potato.'+Var]) | np.isnan(HarvestObsPred.loc[:,'Observed.Potato.'+Var])
    IaNFilter = ~NaNFilter
    Obs = HarvestObsPred.loc[:,'Observed.Potato.'+Var].loc[IaNFilter].values
    Pred = HarvestObsPred.loc[:,'Predicted.Potato.'+Var].loc[IaNFilter].values
    Graph.add_subplot(3,3,Pos)
    ColPos = 0
    MarPos = 0
    for Fol in Folders:
        SimIDs = Factors.loc[Factors.FolderName == Fol].SimulationID.values
        ExpObs = HarvestObsPred.loc[SimIDs,'Observed.Potato.'+Var]
        ExpPred = HarvestObsPred.loc[SimIDs,'Predicted.Potato.'+Var]
        plt.plot(ExpObs,ExpPred,Markers[MarPos],color = Colors[ColPos],label=Fol)
        ColPos +=1
        if ColPos > 16:
            ColPos = 1
            MarPos +=1
    uplim = max(Obs.max(),Pred.max())*1.1
    lowlim = min(Obs.min(),Pred.min())*0.95
    plt.title(Names[Pos-1] + ' (' + Units[Pos-1] + ')')
    plt.ylim(0,uplim)
    plt.xlim(0,uplim)
    plt.plot([lowlim,uplim*.95],[lowlim,uplim*.95],'-')
    RegStats = MUte.MathUtilities.CalcRegressionStats(Var,Pred,Obs)
    LabelText = MakeLabel(RegStats)
    plt.text(uplim*0.05,uplim*.78,LabelText)
    #Fit linear regression to current series and store slope and intercept in dataframe
    ModFit = sm.regression.linear_model.OLS(Pred,  # Y variable
                                        sm.add_constant(Obs), # X variable
                                        missing='drop',                                     # ignor and data where one value is missing
                                        hasconst=False) 
    RegFit = ModFit.fit();  # fit models parameters
    Slope = RegFit.params[1] 
    Intercept = RegFit.params[0]
    Xs = [lowlim,uplim*.95]
    Ys = [Intercept + Xs[0]*Slope,Intercept + Xs[1]*Slope]
    plt.plot(Xs,Ys,'--')
    Pos +=1
plt.text(70,-50,'Observed',fontsize=20)
plt.text(-420,580,'Predicted',fontsize=20,rotation=90)
plt.legend(loc = (1.2,0),ncol=1,numpoints=1,markerscale=2)

In [ ]:
DailyObsPred = pd.read_sql("Select * from DailyObsPred",
                        con)

In [ ]:
RegStats = MUte.MathUtilities.CalcRegressionStats('Test',Obs,Pred)

In [ ]:
def add_subplot_axes(ax,rect,uplim,axisbg='w'):
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3] 
    subax = fig.add_axes([x,y,width,height],axisbg=axisbg,alpha=0.0)
#     x_labelsize = subax.get_xticklabels()[0].get_size()
#     y_labelsize = subax.get_yticklabels()[0].get_size()
#     x_labelsize *= rect[2]**0.5
#     y_labelsize *= rect[3]**0.5
#     subax.xaxis.set_tick_params(labelsize=x_labelsize)
#     subax.yaxis.set_tick_params(labelsize=y_labelsize)
#     #subax.ylim(0,uplim)
    #subax.xlim(0,uplim)
    subax.spines['left'].set_visible(False)
    subax.spines['top'].set_visible(False)
    subax.yaxis.set_ticks_position('left')
    subax.xaxis.set_ticks_position('bottom')
    subax.patch.set_alpha(0)
    subax.xaxis.major.locator.set_params(nbins=2) 
    subax.yaxis.major.locator.set_params(nbins=2)
    return subax

In [ ]:
#initialise graph bits
Graph = plt.figure(figsize=(15,15))
axes = []
insets = []
rect = [0.55, 0.1, .45, .45]
Variables = ['AboveGround.Wt','Grain.Number','Grain.Wt',
             'AboveGround.N','Grain.N','Grain.Protein',
            'Phenology.FlagLeafDAS.Value()','Phenology.FloweringDAS.Value()']
Units = ['$g/m^2$','$/m^2$','$g/m^2$',
        '$g/m^2$','$g/m^2$','%',
        'days','days']
Names = ['Biomass','Grain Number', 'Yield', 'Biomass N', 'Grain N',
         'Grain Protein', 'Flag Leaf','Anthesis']
Pos = 1
#Step through each variable to be graphed
for Var in Variables:
    #Add a subplot
    axes.append(Graph.add_subplot(3,3,Pos))
    #Set up the Data to be graphed
    NaNFilter = np.isnan(HarvestObsPred.ix[:,'Predicted.Wheat.'+Var]) | np.isnan(HarvestObsPred.ix[:,'Observed.Wheat.'+Var])
    IaNFilter = ~NaNFilter
    Obs = HarvestObsPred.ix[:,'Observed.Wheat.'+Var].ix[IaNFilter].values
    Pred = HarvestObsPred.ix[:,'Predicted.Wheat.'+Var].ix[IaNFilter].values
    #Derive some things from the data
    uplim = max(Obs.max(),Pred.max())*1.1
    lowlim = min(Obs.min(),Pred.min())*0.95
    RegStats = MUte.MathUtilities.CalcRegressionStats(Var,Pred,Obs)
    LabelText = MakeLabel(RegStats)
    plt.text(-1.4,0.9,LabelText)
    #Step through each experiment in the data set
    ColPos = 0
    ColPos2 = 0
    MarPos = 0
    MarPos2 = 0
    for Exp in Experiments:
        #Select data for the experiment and calculate means
        RelObs = []
        RelPred = []
        ObsResp = []
        PredResp = []
        SimIDs = Factors.ix[Factors.ExperimentName == Exp].SimulationID.values
        ExpObs = HarvestObsPred.ix[SimIDs,'Observed.Wheat.'+Var].values
        ObsMean = HarvestObsPred.ix[SimIDs,'Observed.Wheat.'+Var].mean()
        ExpPred = HarvestObsPred.ix[SimIDs,'Predicted.Wheat.'+Var].values
        PredMean = HarvestObsPred.ix[SimIDs,'Predicted.Wheat.'+Var].mean()
        #Calculate relative treatment responses
        for x in range(len(ExpObs)):
            ObsResp.append((ExpObs[x]-ObsMean)/ObsMean)
            PredResp.append((ExpPred[x]-PredMean)/PredMean)
        #Plot relative responses
        plt.plot([-1.2,1.2],[-1.2,1.2],'--',color='g')
        plt.plot(ObsResp,PredResp,Markers[MarPos2],color = Colors[ColPos2],label=Exp)
        axes[Pos-1].spines['right'].set_visible(False)
        axes[Pos-1].spines['top'].set_visible(False)
        plt.tick_params(axis='x', which='both', bottom='on',top='off', labelbottom='on')
        plt.tick_params(axis='y', which='both', left='on',right='off', labelbottom='on')
        plt.title(Names[Pos-1])
        plt.xlim(-1.5,1.5)
        plt.ylim(-1.5,1.5)
        ColPos2 +=1
        if ColPos2 > 17:
            ColPos2 = 1
            MarPos2 +=1
        if Pos in [1,2,3,4,5,6]:
            axes[Pos-1].tick_params(labelbottom='off') 
        if Pos in [2,3,5,6,8,9]:
            axes[Pos-1].tick_params(labelleft='off') 
        if Pos == 4:
            plt.ylabel('Predicted Relative Treatment Response', fontsize =16)
        if Pos == 8: 
            plt.xlabel('Observed Relative Treatment Response', fontsize =16)
            plt.legend(loc = (1.25,-.13),numpoints=1,fontsize=8,ncol=2)
    #Add insert into pannel to graph experimental means
    insets.append(add_subplot_axes(axes[Pos-1],rect,uplim))
    #Step through experiments and add means to inset
    for Exp in Experiments:
        SimIDs = Factors.ix[Factors.ExperimentName == Exp].SimulationID.values
        ExpObs = HarvestObsPred.ix[SimIDs,'Observed.Wheat.'+Var].mean()
        ExpPred = HarvestObsPred.ix[SimIDs,'Predicted.Wheat.'+Var].mean()
        plt.plot([lowlim*1.2,uplim*.9],[lowlim*1.2,uplim*.9],'--',color='b')
        plt.plot(ExpObs,ExpPred,Markers[MarPos],mec = Colors[ColPos],mfc='white',label=Exp,mew=2)
        ColPos +=1
        if ColPos > 17:
            ColPos = 1
            MarPos +=1
    plt.ylim(0,uplim)
    plt.xlim(0,uplim)
    plt.tick_params(axis='x', which='both', bottom='on',top='off', labelbottom='on')
    plt.tick_params(axis='y', which='both', left='off',right='on', labelright='on', labelleft='off')
    plt.text(uplim*0.7,uplim*.05,Units[Pos-1],fontsize = 14)
    Pos +=1
plt.tight_layout 
Graph.savefig('ModelAssessment.jpg',dpi=200,pad_inches=1)

In [ ]:
Experiments

In [ ]:
YieldProfitValidation = pd.read_excel('WheatValidation.xlsx',sheetname='Sheet1')

In [ ]:
YieldProfitValidation.ix[:,'Year'] = YieldProfitValidation.ix[:,'Clock.Today'].dt.year

In [ ]:
YearList = YieldProfitValidation.ix[:,'Year'].dropna().drop_duplicates()

In [ ]:
(YieldProfitValidation.ix[:,'Comments' ])==0

In [ ]:
Graph = plt.figure(figsize=(6,6))
for Year in YearList:
    filter = (YieldProfitValidation.ix[:,'Year'] == Year) & (YieldProfitValidation.ix[:,'Observed Yield']>0) & (YieldProfitValidation.ix[:,'WetYield']>0) &(YieldProfitValidation.ix[:,'Comments' ]==0)
    plt.plot(YieldProfitValidation.ix[filter,'Observed Yield']/100,
            YieldProfitValidation.ix[filter, 'WetYield']/100,
            'o')

filter = (YieldProfitValidation.ix[:,'Observed Yield']>0) & (YieldProfitValidation.ix[:,'WetYield']>0) &(YieldProfitValidation.ix[:,'Comments' ]==0)
Pred = YieldProfitValidation.ix[filter, 'WetYield']/100
Obs = YieldProfitValidation.ix[filter,'Observed Yield']/100
RegStats = MUte.MathUtilities.CalcRegressionStats('Grain.Wt',Pred.values,Obs.values)
plt.ylim(0,9)
plt.xlim(0,9)
plt.plot([0,9],[0,9],'-',color='k',linewidth=4)
stats = 'y = ' +'%.2f'%RegStats.Intercept + ' + ' + '%.2f'%RegStats.Slope + 'x \n$R^2$ = ' + '%.2f'%RegStats.R2
plt.text(0.5,7.8,stats)
plt.plot([0,9],[RegStats.Intercept,RegStats.Intercept/100+900*RegStats.Slope/100],
        '--',color='k',linewidth = 4)
plt.xlabel('Observed Yield (t/ha)')
plt.ylabel('Predicted Yield (t/ha)')

In [ ]:
Pred = YieldProfitValidation.ix[filter, 'WetYield']
Obs = YieldProfitValidation.ix[filter,'Observed Yield']
test = MUte.MathUtilities.CalcRegressionStats('Grain.Wt',Pred.values,Obs.values)

In [ ]:
'%.2f'%test.Intercept

In [ ]:
test.Slope

In [ ]:
test.R2

In [ ]:
Obs.size